# Credit Card Churn

By Eric Wilson

The individual on Kaggle who submitted this data set said they need to predict customer churn, and have managed to get 62% as the highest accuracy. It's ok to predict someone who will stay as one who will churn, but the most important task is making sure everyone who will churn is not marked as someone who will stay. Let's see what we can do...

### Import libraries and data

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
df = pd.read_csv('../input/credit-card-customers/BankChurners.csv')
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


First, we need to turn attrition/churn into numeric values, followed by gender, education, income, marital status, and card catagory. We also need to get rid of the last two columns.

In [3]:
print(df['Attrition_Flag'].value_counts())
print(df['Gender'].value_counts())
print(df['Education_Level'].value_counts())
print(df['Marital_Status'].value_counts())
print(df['Income_Category'].value_counts())
print(df['Card_Category'].value_counts())

Existing Customer    8500
Attrited Customer    1627
Name: Attrition_Flag, dtype: int64
F    5358
M    4769
Name: Gender, dtype: int64
Graduate         3128
High School      2013
Unknown          1519
Uneducated       1487
College          1013
Post-Graduate     516
Doctorate         451
Name: Education_Level, dtype: int64
Married     4687
Single      3943
Unknown      749
Divorced     748
Name: Marital_Status, dtype: int64
Less than $40K    3561
$40K - $60K       1790
$80K - $120K      1535
$60K - $80K       1402
Unknown           1112
$120K +            727
Name: Income_Category, dtype: int64
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: Card_Category, dtype: int64


In [4]:
df['Attrition_Flag'].replace({'Existing Customer' : 0, 'Attrited Customer' : 1},inplace = True)
df['Gender'].replace({'F': 0, 'M': 1}, inplace = True)
df['Education_Level'].replace({'Unknown' : 0, 'Uneducated' : 1, 'High School' : 2, 'College' : 3, 
                               'Graduate' : 4, 'Post-Graduate' : 5, 'Doctorate' : 6}, inplace = True)
df['Marital_Status'].replace({'Unknown' : 0, 'Single' : 1, 'Divorced' : 2, 'Married' : 3}, inplace = True)
df['Income_Category'].replace({'Unknown' : 0, 'Less than $40K' : 1, '$40K - $60K' : 2, '$60K - $80K' : 3,
                              '$80K - $120K' : 4, '$120K +' : 5}, inplace = True)
df['Card_Category'].replace({'Blue' : 0, 'Silver' : 1, 'Gold' : 2, 'Platinum' : 3}, inplace = True)
df.drop(df.columns[[0,21,22]].values,axis=1,inplace = True)
df.dtypes

Attrition_Flag                int64
Customer_Age                  int64
Gender                        int64
Dependent_count               int64
Education_Level               int64
Marital_Status                int64
Income_Category               int64
Card_Category                 int64
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
dtype: object

In [5]:
df.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,1,3,2,3,3,0,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,0,5,4,1,1,0,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,1,3,4,3,4,0,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,0,4,2,0,1,0,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,1,3,1,3,3,0,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


Now we have nothing but numbers. Let's start trying to build a model.

### Correlation

Let's start by seeing what features correlate most with Attrition, and which correlate with one another, in order to have an idea of what features may be more useful than others in an attempt to avoid data overload and overfitting.

In [6]:
df.corr()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
Attrition_Flag,1.000000,0.018203,-0.037272,0.018991,0.008796,-0.024117,-0.013577,0.002354,0.013687,-0.150005,0.152449,0.204491,-0.023873,-0.263053,-0.000285,-0.131063,-0.168598,-0.371403,-0.290054,-0.178410
Customer_Age,0.018203,1.000000,-0.017312,-0.122254,-0.002369,0.040298,0.023508,-0.018235,0.788912,-0.010931,0.054361,-0.018452,0.002476,0.014780,0.001151,-0.062042,-0.046446,-0.067097,-0.012143,0.007114
Gender,-0.037272,-0.017312,1.000000,0.004563,-0.005087,0.006491,0.786608,0.080093,-0.006728,0.003157,-0.011163,0.039987,0.420806,0.029658,0.418059,0.026712,0.024890,-0.067454,-0.005800,-0.257851
Dependent_count,0.018991,-0.122254,0.004563,1.000000,0.000472,0.004839,0.066278,0.030469,-0.103062,-0.039076,-0.010768,-0.040505,0.068065,-0.002688,0.068291,-0.035439,0.025046,0.049912,0.011087,-0.037135
Education_Level,0.008796,-0.002369,-0.005087,0.000472,1.000000,0.015478,-0.011677,0.014989,0.006613,0.000766,0.005761,-0.006280,-0.002354,-0.006800,-0.001743,-0.010040,-0.007460,-0.004307,-0.016692,-0.001849
Marital_Status,-0.024117,0.040298,0.006491,0.004839,0.015478,1.000000,0.017770,-0.052296,0.030598,0.020427,-0.004915,0.001780,-0.053580,0.036352,-0.056828,0.050805,-0.062698,-0.116268,0.008999,0.046623
Income_Category,-0.013577,0.023508,0.786608,0.066278,-0.011677,0.017770,1.000000,0.077326,0.022122,-0.003202,-0.016310,0.023113,0.475972,0.034718,0.472760,0.011352,0.019651,-0.054569,-0.012657,-0.246476
Card_Category,0.002354,-0.018235,0.080093,0.030469,0.014989,-0.052296,0.077326,1.000000,-0.012535,-0.094077,-0.014629,-0.000442,0.492446,0.026304,0.489985,0.007385,0.196003,0.134275,-0.007261,-0.198711
Months_on_book,0.013687,0.788912,-0.006728,-0.103062,0.006613,0.030598,0.022122,-0.012535,1.000000,-0.009203,0.074164,-0.010774,0.007507,0.008623,0.006732,-0.048959,-0.038591,-0.049819,-0.014072,-0.007541
Total_Relationship_Count,-0.150005,-0.010931,0.003157,-0.039076,0.000766,0.020427,-0.003202,-0.094077,-0.009203,1.000000,-0.003675,0.055203,-0.071386,0.013726,-0.072601,0.050119,-0.347229,-0.241891,0.040831,0.067663


It appears that the most correlated fields to churn are transaction counts, count change from Q4 to Q1, revolving balance, 12 month contact count, inactive months, utilization ratio,relationship count, and transaction amount. That being said, none of them share a particularly strong correlation, but essentially all of the demographic information (gender, income, education level, marital status, dependant count) lack any real correlation with churn.

### Model Building

In [7]:
dfm = df[['Attrition_Flag', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
          'Contacts_Count_12_mon', 'Total_Revolving_Bal', 'Total_Trans_Amt', 'Total_Trans_Ct',
          'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']]
dfm.corr()

,Attrition_Flag,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
Attrition_Flag,1.000000,-0.150005,0.152449,0.204491,-0.263053,-0.168598,-0.371403,-0.290054,-0.178410
Total_Relationship_Count,-0.150005,1.000000,-0.003675,0.055203,0.013726,-0.347229,-0.241891,0.040831,0.067663
Months_Inactive_12_mon,0.152449,-0.003675,1.000000,0.029493,-0.042210,-0.036982,-0.042787,-0.038989,-0.007503
Contacts_Count_12_mon,0.204491,0.055203,0.029493,1.000000,-0.053913,-0.112774,-0.152213,-0.094997,-0.055471
Total_Revolving_Bal,-0.263053,0.013726,-0.042210,-0.053913,1.000000,0.064370,0.056060,0.089861,0.624022
Total_Trans_Amt,-0.168598,-0.347229,-0.036982,-0.112774,0.064370,1.000000,0.807192,0.085581,-0.083034
Total_Trans_Ct,-0.371403,-0.241891,-0.042787,-0.152213,0.056060,0.807192,1.000000,0.112324,0.002838
Total_Ct_Chng_Q4_Q1,-0.290054,0.040831,-0.038989,-0.094997,0.089861,0.085581,0.112324,1.000000,0.074143
Avg_Utilization_Ratio,-0.178410,0.067663,-0.007503,-0.055471,0.624022,-0.083034,0.002838,0.074143,1.000000


In [8]:
x = dfm[['Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
        'Total_Revolving_Bal', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1',
        'Avg_Utilization_Ratio']]
y = dfm['Attrition_Flag']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2)

In [9]:
model = RandomForestClassifier(n_estimators=100, max_depth=13, random_state=2)
model.fit(x, y)
rfvalue = model.predict(x_test)

print('Model Accuracy : ', accuracy_score(y_test, rfvalue) *  100)
print('Model Recall : ', recall_score(y_test, rfvalue) *  100)
print('Model Precision : ', precision_score(y_test, rfvalue) *  100)

Model Accuracy :  99.11154985192498
Model Recall :  95.23809523809523
Model Precision :  99.13793103448276


In [10]:
print(confusion_matrix(y_test, rfvalue))
print(classification_report(y_test, rfvalue))

[[2552    4]
 [  23  460]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2556
           1       0.99      0.95      0.97       483

    accuracy                           0.99      3039
   macro avg       0.99      0.98      0.98      3039
weighted avg       0.99      0.99      0.99      3039



With this model, we still have roughly two dozen false negatives - what we've been asked to avoid. That being said, we're still looking at pretty high accuracy, precision, recall, and f1 scores.

## Conclusion

By narrowing down the data used to the factors which have the highest correlation to attrition, we're left with a pretty accurate model. I've tried to optimize it with larger and smaller train / test splits and random states, but the combination used in this notebook seemed to be pretty optimal.

### Addendum

I value feedback, tips, and criticism highly - I'm still fairly new to DS and ML, so if I make a mistake or error, I would greatly appreciate knowing so; the best way to learn is by doing, and it's better to fix an error before it becomes a habit.

Thank you for taking the time to read this notebook!